In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# 定義梯度下降批量
batch_size = 128
# 定義分類數量
num_classes = 10
# 定義訓練週期
epochs = 12

# 定義圖像寬、高
img_rows, img_cols = 28, 28

# 載入 MNIST 訓練資料
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 保留原始資料，供 cross tab function 使用
y_test_org = y_test

In [4]:
import numpy as np
print(np.shape(x_train))

(60000, 28, 28, 1)


In [6]:
print(K.image_data_format())

channels_last


In [3]:
# channels_first: 色彩通道(R/G/B)資料(深度)放在第2維度，第3、4維度放置寬與高
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)  #1表示灰階高度
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else: # channels_last: 色彩通道(R/G/B)資料(深度)放在第4維度，第2、3維度放置寬與高
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [7]:
print(np.shape(x_train))


(60000, 28, 28, 1)
60000


In [8]:
# 轉換色彩 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
# y 值轉成 one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(64, (3, 3), activation='relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))
# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())
# 全連接層: 128個output
model.add(Dense(128, activation='relu'))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
model.add(Dropout(0.5))
# 使用 softmax activation function，將結果分類
model.add(Dense(num_classes, activation='softmax'))


In [11]:
# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [12]:
# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

#一個epoch要跑  訓練樣本數/batch_size個iterations
#因為一個 Iteration 是指一次用Batch_Size 個樣本做整個網路的導數計算來更新參數的

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 11s 179us/step - loss: 0.2673 - acc: 0.9180 - val_loss: 0.0590 - val_acc: 0.9803
Epoch 2/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0877 - acc: 0.9744 - val_loss: 0.0379 - val_acc: 0.9872
Epoch 3/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0651 - acc: 0.9803 - val_loss: 0.0352 - val_acc: 0.9882
Epoch 4/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0535 - acc: 0.9838 - val_loss: 0.0320 - val_acc: 0.9893
Epoch 5/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0466 - acc: 0.9861 - val_loss: 0.0298 - val_acc: 0.9901
Epoch 6/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0409 - acc: 0.9875 - val_loss: 0.0262 - val_acc: 0.9912
Epoch 7/12
60000/60000 [==============================] - 6s 104us/step - loss: 0.0370 - acc: 0.9884 - val_loss: 0.0242 -

In [13]:
# 計算『混淆矩陣』(Confusion Matrix)，顯示測試集分類的正確及錯認總和數
import pandas as pd 
predictions = model.predict_classes(x_test) 
pd.crosstab(y_test_org, predictions, rownames=['實際值'], colnames=['預測值'])

預測值,0,1,2,3,4,5,6,7,8,9
實際值,,,,,,,,,,
0,977,1,0,0,0,0,1,1,0,0
1,0,1133,1,0,0,0,0,1,0,0
2,1,1,1023,0,1,0,1,5,0,0
3,0,0,3,1000,0,2,0,2,2,1
4,0,0,0,0,980,0,1,0,0,1
5,2,0,0,3,0,884,3,0,0,0
6,3,2,0,0,1,1,950,0,1,0
7,0,2,5,0,0,0,0,1019,1,1
8,4,1,1,1,2,0,1,1,960,3


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________